In [2]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

!pip install bs4
from bs4 import BeautifulSoup
import requests

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=3f4f19860e9886d3e7b05f74b9d8a8e6be2e8df531e7420bda82df1a69145039
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Libraries imported.


In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"

In [60]:
data = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
#print(soup.prettify())

In [61]:
tables = soup.find_all('table')

In [62]:
len(tables)

3

Expecting index to be 0

In [63]:
for index, table in enumerate(tables):
    if ("Postal Code" in str(table)):
        table_index = index
print(table_index)

0


View HTML data

In [65]:
#print(tables[table_index].prettify())

Writting data from HTML into dataframe

In [66]:
postal_code_data = pd.DataFrame(columns=["Postal Code","Borough","Neighbourhood"])

for row in tables[0].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        postal_code = col[0].text
        borough = col[1].text
        neighbourhood = col[2].text
        postal_code_data = postal_code_data.append({"Postal Code":postal_code, "Borough":borough, "Neighbourhood":neighbourhood}, ignore_index=True)

postal_code_data.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Clean up by removing "\n" at the end of every cell

In [67]:
postal_code_data = postal_code_data.replace('\\n','',regex=True)
postal_code_data.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Cleaning up data (remove "Not assigned") and reset index

In [68]:
postal_code_data = postal_code_data[postal_code_data["Borough"] != "Not assigned"]
postal_code_data = postal_code_data[postal_code_data["Neighbourhood"] != "Not assigned"]
postal_code_data.reset_index(drop=True, inplace=True)
postal_code_data.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Getting dimensions of data frame for report

In [69]:
postal_code_data.shape

(103, 3)